# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [13]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('c:/Tina Lin/Training/DSI/Week10_Production/assignments/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [15]:

from sklearn.model_selection import train_test_split

# Create DataFrame X with features (all columns except 'income')
X = adult_dt.drop(columns='income')

# Create DataFrame Y with target data (only the 'income' column)
Y = adult_dt['income']

# Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Print the shapes of the resulting datasets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)
print("Training target shape:", Y_train.shape)
print("Testing target shape:", Y_test.shape)


Training set shape: (22792, 14)
Testing set shape: (9769, 14)
Training target shape: (22792,)
Testing target shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 

[Answer] The random state parameter in functions like train_test_split in scikit-learn is used to control the randomness of the data splitting process. You can set random_state to any integer value. Common practice is to use simple numbers like 0, 42, or any number you choose.

+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

[Answer] The random_state parameter is vital for ensuring reproducibility, facilitating reliable evaluations, and maintaining consistency in experiments. This makes it a fundamental practice in data science and machine learning workflows, enabling data scientists and researchers to produce reliable and valid results that can be trusted and replicated by others.

*(Comment here.)*

Question1: [Answer] The random state parameter in functions like train_test_split in scikit-learn is used to control the randomness of the data splitting process. You can set random_state to any integer value. Common practice is to use simple numbers like 0, 42, or any number you choose.

Question2: [Answer] The random_state parameter is vital for ensuring reproducibility, facilitating reliable evaluations, and maintaining consistency in experiments. This makes it a fundamental practice in data science and machine learning workflows, enabling data scientists and researchers to produce reliable and valid results that can be trusted and replicated by others.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [18]:

from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline  # Import Pipeline

# Identify numerical and categorical columns
numerical_cols = [
    'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]

categorical_cols = [
    'workclass', 'education', 'marital-status', 'occupation',
    'relationship', 'race', 'sex', 'native-country'
]

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
             ('scaler', RobustScaler())
         ]), 
         numerical_cols),
        
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')),
             ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
         ]), 
         categorical_cols)
    ]
)

# Use this preprocessor to fit and transform your data

X_transformed = preprocessor.fit_transform(X)

# Convert the transformed data back to a DataFrame
X_transformed_df = pd.DataFrame(X_transformed)
print(X_transformed_df.head())


                                                   0
0  <Compressed Sparse Row sparse matrix of dtype ...
1  <Compressed Sparse Row sparse matrix of dtype ...
2  <Compressed Sparse Row sparse matrix of dtype ...
3  <Compressed Sparse Row sparse matrix of dtype ...
4  <Compressed Sparse Row sparse matrix of dtype ...


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [19]:
#from sklearn.compose import ColumnTransformer
#from sklearn.impute import KNNImputer, SimpleImputer
#from sklearn.preprocessing import OneHotEncoder, RobustScaler
#from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Identify numerical and categorical columns
numerical_cols = [
    'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]

categorical_cols = [
    'workclass', 'education', 'marital-status', 'occupation',
    'relationship', 'race', 'sex', 'native-country'
]

# Create the ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
             ('scaler', RobustScaler())
         ]), 
         numerical_cols),
        
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')),
             ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
         ]), 
         categorical_cols)
    ]
)

# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Fit the model pipeline to the training data
# Split the data into training and testing sets
X = adult_dt.drop(columns='income')
Y = adult_dt['income']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Fit the pipeline to the training data
model_pipeline.fit(X_train, Y_train)

# You can now use model_pipeline to make predictions on the test set
Y_pred = model_pipeline.predict(X_test)

# Optionally, print the predictions
print(Y_pred)

[0 0 1 ... 0 1 1]


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [20]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Identify numerical and categorical columns
numerical_cols = [
    'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]

categorical_cols = [
    'workclass', 'education', 'marital-status', 'occupation',
    'relationship', 'race', 'sex', 'native-country'
]

# Create the ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
             ('scaler', RobustScaler())
         ]), 
         numerical_cols),
        
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')),
             ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
         ]), 
         categorical_cols)
    ]
)

# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Split the data into features and target
X = adult_dt.drop(columns='income')
Y = adult_dt['income']

# Evaluate the model pipeline using cross-validation
scoring = {
    'neg_log_loss': make_scorer(log_loss, needs_proba=True),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

# Perform cross-validation with 5 folds
cv_results = cross_validate(model_pipeline, X, Y, cv=5, scoring=scoring, return_train_score=True)

# Report the training and validation results
print("Training Results:")
print(f"Negative Log Loss: {cv_results['train_neg_log_loss'].mean():.4f} ± {cv_results['train_neg_log_loss'].std():.4f}")
print(f"ROC AUC: {cv_results['train_roc_auc'].mean():.4f} ± {cv_results['train_roc_auc'].std():.4f}")
print(f"Accuracy: {cv_results['train_accuracy'].mean():.4f} ± {cv_results['train_accuracy'].std():.4f}")
print(f"Balanced Accuracy: {cv_results['train_balanced_accuracy'].mean():.4f} ± {cv_results['train_balanced_accuracy'].std():.4f}")

print("\nValidation Results:")
print(f"Negative Log Loss: {cv_results['test_neg_log_loss'].mean():.4f} ± {cv_results['test_neg_log_loss'].std():.4f}")
print(f"ROC AUC: {cv_results['test_roc_auc'].mean():.4f} ± {cv_results['test_roc_auc'].std():.4f}")
print(f"Accuracy: {cv_results['test_accuracy'].mean():.4f} ± {cv_results['test_accuracy'].std():.4f}")
print(f"Balanced Accuracy: {cv_results['test_balanced_accuracy'].mean():.4f} ± {cv_results['test_balanced_accuracy'].std():.4f}")




c:\Users\ryanw\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
c:\Users\ryanw\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\ryanw\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training Results:
Negative Log Loss: 0.0803 ± 0.0005
ROC AUC: 1.0000 ± 0.0000
Accuracy: 1.0000 ± 0.0000
Balanced Accuracy: 1.0000 ± 0.0000

Validation Results:
Negative Log Loss: 0.3649 ± 0.0205
ROC AUC: 0.7749 ± 0.0052
Accuracy: 0.8542 ± 0.0034
Balanced Accuracy: 0.7749 ± 0.0052


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [21]:
# Create a DataFrame for fold-level results
fold_results = pd.DataFrame({
    'Fold': range(1, 6),
    'Negative Log Loss': cv_results['test_neg_log_loss'],
    'ROC AUC': cv_results['test_roc_auc'],
    'Accuracy': cv_results['test_accuracy'],
    'Balanced Accuracy': cv_results['test_balanced_accuracy']
})

# Sort the DataFrame by Negative Log Loss
fold_results_sorted = fold_results.sort_values(by='Negative Log Loss')

# Display the sorted fold-level results
print(fold_results_sorted)

   Fold  Negative Log Loss   ROC AUC  Accuracy  Balanced Accuracy
2     3           0.348129  0.778899  0.852733           0.778899
4     5           0.350558  0.780176  0.857647           0.780176
3     4           0.358425  0.777983  0.857955           0.777983
0     1           0.362914  0.770252  0.853984           0.770252
1     2           0.404425  0.767124  0.848741           0.767124


Calculate the mean of each metric. 

In [23]:
# Calculate mean of each metric
mean_results = fold_results_sorted[['Negative Log Loss', 'ROC AUC', 'Accuracy', 'Balanced Accuracy']].mean()

# Display the sorted fold-level results and mean metrics
#print(fold_results_sorted)
print("\nMean of Each Metric:")
print(mean_results)


Mean of Each Metric:
Negative Log Loss    0.364890
ROC AUC              0.774887
Accuracy             0.854212
Balanced Accuracy    0.774887
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [24]:
# Identify numerical and categorical columns
numerical_cols = [
    'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]

categorical_cols = [
    'workclass', 'education', 'marital-status', 'occupation',
    'relationship', 'race', 'sex', 'native-country'
]

# Create the ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
             ('scaler', RobustScaler())
         ]), 
         numerical_cols),
        
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')),
             ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
         ]), 
         categorical_cols)
    ]
)

# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Split the data into features and target
X = adult_dt.drop(columns='income')
Y = adult_dt['income']

# Split the data into training and testing sets (70-30 split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Fit the model pipeline to the training data
model_pipeline.fit(X_train, Y_train)

# Make predictions and get predicted probabilities
Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate the performance metrics
performance_metrics = {
    'Negative Log Loss': log_loss(Y_test, Y_pred_proba),
    'ROC AUC': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'Accuracy': accuracy_score(Y_test, Y_pred),
    'Balanced Accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

# Display the results as a dictionary
print("Performance Metrics on Testing Data:")
print(performance_metrics)

Performance Metrics on Testing Data:
{'Negative Log Loss': 0.38733674303408167, 'ROC AUC': np.float64(0.9018825137514803), 'Accuracy': 0.8564847988535162, 'Balanced Accuracy': np.float64(0.7784191753807199)}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

[Answer] Recoding the target variable upon loading the data is a best practice that streamlines the data preprocessing pipeline, making it easier to work with the data and ensuring compatibility with modeling tools and functions. It enhances both the efficiency of the workflow and the clarity of the code.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.